In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split
import os

In [3]:
df = pd.read_excel("loan_processed.xlsx")

In [25]:
def cleanmessage(string):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    string = string.lower()
    string = nltk.word_tokenize(string)
    string = [w for w in string if w.isalnum()]
    string = [lemmatizer.lemmatize(w) for w in string]
    string = ' '.join(string)
    return string

In [ ]:
cleaned_list = []
count = 0
for string in df["desc_full"]:
    print(count)
    count+=1
    cleaned_string = re.sub(r"<br>", "", string)
    cleaned_string = re.sub(r"Borrower added on [0-9]+/[0-9]+/[0-9]+ >", "", cleaned_string)
    cleaned_string = re.sub(r"[0-9]+", "number", cleaned_string)
    regex = re.compile('[^a-zA-Z ]')
    cleaned_string = regex.sub('', cleaned_string)

    text_tokens = word_tokenize(cleaned_string)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    cleaned_string = " ".join(tokens_without_sw)
    cleaned_list.append(cleanmessage(cleaned_string))
            
df["cleaned_desc"] = cleaned_list

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df, df["loan_status"], test_size=0.1, random_state=42)

In [19]:

document_class = {1: [],
 0: []
}

for index, row in X_train.iterrows():
    document_class[row["loan_status"]].append(row["cleaned_desc"])
        

for key in document_class:
    try:
        os.makedirs("loan_status/{}".format(key))
    except FileExistsError:
        pass
    count = 0
    for i in document_class[key]:
        textfile = open("loan_status/{}/{}_file{}.txt".format(key, key, count), "w")
        textfile.write(str(i))
        textfile.close()
        count += 1

In [20]:

document_class = {1: [],
 0: []
}

for index, row in X_test.iterrows():
    document_class[row["loan_status"]].append(row["cleaned_desc"])
        

for key in document_class:
    try:
        os.makedirs("loan_status_eval/{}".format(key))
    except FileExistsError:
        pass
    count = 0
    for i in document_class[key]:
        textfile = open("loan_status_eval/{}/{}_file{}.txt".format(key, key, count), "w")
        textfile.write(str(i))
        textfile.close()
        count += 1

In [2]:
df = pd.read_excel("loan_processed_cleaned.xlsx")

In [7]:
df_important = pd.read_excel("loan_data_filtered_important.xlsx")

In [3]:
from textblob import TextBlob

In [4]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [11]:
subjectivity = []
polarity = []

for text in df["cleaned_desc"]:
    try:
        subjectivity.append(getSubjectivity(text))
        polarity.append(getPolarity(text))
    except:
        subjectivity.append(0)
        polarity.append(0)

In [12]:
df_important["polarity"] = polarity
df_important["subjectivity"] = subjectivity

In [13]:
df_important.to_excel("loan_data_final.xlsx", index = False)